Import the required libraries

In [1]:
from d3rlpy.datasets import get_cartpole
from d3rlpy.algos import DiscreteCQL, DQN
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.dataset import Episode
from d3rlpy.dataset import MDPDataset

from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split

import import_ipynb
import numpy as np
from random import random
from create_dataset import CreateDataset
from FootballEnv import FootballEnv

importing Jupyter notebook from FootballEnv.ipynb
(12, 16)
(5,)


Helper function to create a dummy dataset

In [6]:
def create_dataset():

    dataset_maker = CreateDataset()
    dataset_maker.loadFile('data.json')

    observations, actions, rewards = dataset_maker.createEpisodeDataset()

    # observations = np.array([[int(random() * 5) for i in range(5)] for i in range(5000)])
    # actions = np.array([int(random() * 5) for i in range(5000)])
    # rewards = np.array([ random() for i in range(5000)])
    terminals = np.array([[0, 0, 0, 0, 1] for i in range(len(observations))])

    return MDPDataset(
        observations,
        actions,
        rewards, 
        terminals,
    )

In [7]:
dataset = create_dataset()
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

(12, 16)


In [11]:
# setup CQL algorithm
cql = DiscreteCQL(use_gpu=False)

env = FootballEnv()

# start training
output = cql.fit(
    train_episodes,
    eval_episodes=test_episodes,
    n_epochs=50,
    scorers={
        'environment': evaluate_on_environment(env), # evaluate with Football Env
        'advantage': discounted_sum_of_advantage_scorer, # smaller is better
        'td_error': td_error_scorer, # smaller is better
        'value_scale': average_value_estimation_scorer # smaller is better
    }
)

(12, 16)
2022-02-06 19:40.04 [debug    ] RoundIterator is selected.
2022-02-06 19:40.04 [info     ] Directory is created at d3rlpy_logs\DiscreteCQL_20220206194004
2022-02-06 19:40.04 [debug    ] Building models...
2022-02-06 19:40.04 [debug    ] Models have been built.
2022-02-06 19:40.04 [info     ] Parameters are saved to d3rlpy_logs\DiscreteCQL_20220206194004\params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'bootstrap': False, 'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_reduction_type': 'min', 'target_update_interval': 80

Epoch 1/50: 100%|██████████| 39/39 [00:00<00:00, 117.42it/s, loss=0.97]


2022-02-06 19:40.05 [info     ] DiscreteCQL_20220206194004: epoch=1 step=39 epoch=1 metrics={'time_sample_batch': 9.464606260642027e-05, 'time_algorithm_update': 0.008133221895266801, 'loss': 0.9401729412567921, 'time_step': 0.008381085518078927, 'environment': 2.776230746130838, 'advantage': 0.0, 'td_error': 0.03976752685707652, 'value_scale': 0.39979344606399536} step=39
2022-02-06 19:40.05 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220206194004\model_39.pt


Epoch 2/50: 100%|██████████| 39/39 [00:00<00:00, 121.08it/s, loss=0.737]


2022-02-06 19:40.06 [info     ] DiscreteCQL_20220206194004: epoch=2 step=78 epoch=2 metrics={'time_sample_batch': 0.00023157779987041766, 'time_algorithm_update': 0.007747919131547977, 'loss': 0.7230974145424671, 'time_step': 0.008157045413286258, 'environment': 2.2611585221115154, 'advantage': 0.0, 'td_error': 0.0549387208875487, 'value_scale': 0.4698936939239502} step=78
2022-02-06 19:40.06 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220206194004\model_78.pt


...

2022-02-06 19:40.40 [info     ] DiscreteCQL_20220206194004: epoch=48 step=1872 epoch=48 metrics={'time_sample_batch': 0.00023094813028971353, 'time_algorithm_update': 0.00700221917568109, 'loss': 0.0008968108106786624, 'time_step': 0.007489638450818184, 'environment': 2.522024862091464, 'advantage': 0.0, 'td_error': 0.0007024166164200096, 'value_scale': 0.04597160220146179} step=1872
2022-02-06 19:40.40 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220206194004\model_1872.pt


Epoch 49/50: 100%|██████████| 39/39 [00:00<00:00, 134.27it/s, loss=0.00087] 


2022-02-06 19:40.40 [info     ] DiscreteCQL_20220206194004: epoch=49 step=1911 epoch=49 metrics={'time_sample_batch': 0.0001763441623785557, 'time_algorithm_update': 0.007068933584751227, 'loss': 0.0008589333633725077, 'time_step': 0.007398140736115284, 'environment': 2.735316768516915, 'advantage': 0.0, 'td_error': 0.0005257380115324395, 'value_scale': 0.03699507191777229} step=1911
2022-02-06 19:40.40 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220206194004\model_1911.pt


Epoch 50/50: 100%|██████████| 39/39 [00:00<00:00, 139.78it/s, loss=0.00084] 


2022-02-06 19:40.41 [info     ] DiscreteCQL_20220206194004: epoch=50 step=1950 epoch=50 metrics={'time_sample_batch': 0.00015287521557930188, 'time_algorithm_update': 0.006796971345559144, 'loss': 0.0008299540779075753, 'time_step': 0.007103608204768254, 'environment': 2.703058760858339, 'advantage': 0.0, 'td_error': 0.0004425350983989773, 'value_scale': 0.03184283897280693} step=1950
2022-02-06 19:40.41 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220206194004\model_1950.pt
